In [1]:
pip install sklearn-crfsuite

  Stored in directory: C:\Users\TK\AppData\Local\pip\Cache\wheels\2b\67\89\414471314a2d15de625d184d8be6d38a03ae1e983dbda91e84
Successfully built tabulate
Note: you may need to restart the kernel to use updated packages.


In [2]:
import nltk
from nltk.corpus import treebank,brown
from nltk import bigrams, ngrams, trigrams
import math
import copy
import sklearn
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
nltk.download('treebank')
nltk.download('brown')
nltk.download('universal_tagset')
nltk_data = list(nltk.corpus.treebank.tagged_sents())

[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\TK\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\TK\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\brown.zip.
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\TK\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\universal_tagset.zip.


In [3]:
corpus = brown.tagged_sents(tagset='universal')[:-100] 

tag_dict={}
word_dict={}

for sent in corpus:
    for elem in sent:
        w = elem[0]
        tag= elem[1]

        if w not in word_dict:
            word_dict[w]=0

        if tag not in tag_dict:
            tag_dict[tag]=0

        word_dict[w]+=1
        tag_dict[tag]+=1

The project does POS tagging on tagged brown corpus using Hidden Markov Model (Viterbi Algorithm) and CRF (Conditional Random Fields)

Before we use the probabilistic methods mentioned above to tag the words, it is necessary for us to do the data preparation such as splitting the data in to Training and Test data. Brown Corpus was compiled in the 1960s by Henry Kučera and W. Nelson Francis at Brown University, Providence, Rhode Island as a general corpus (text collection) in the field of corpus linguistics. It contains 500 samples of English-language text, totaling roughly one million words, compiled from works published in the United States in 1961.



In [4]:
'''-----------------Build Start, Transition and Emission Matrix-----------------'''
start={}
for i in corpus:
    if i[0] not in start:
        start[i[0][1]]= 0
for i in corpus:
    start[i[0][1]]+=1

for i in start:
    start[i]=math.log2(start[i]/len(corpus))
    
transition={}
transition1 = {}
for i in corpus:
    for j in range(len(i)-1):
        if i[j][1] not in transition1:
            transition1[i[j][1]]={}
        if i[j+1][1] not in transition1[i[j][1]]:
            transition1[i[j][1]][i[j+1][1]]=0
        transition1[i[j][1]][i[j+1][1]]+=1
transition= copy.deepcopy(transition1)        
for w1 in transition:
    tot = float(sum(transition[w1].values()))
    for w2 in transition[w1]:
        transition[w1][w2]=math.log2((0.001+ transition[w1][w2])/(0.001*len(word_dict) + tot))
        
transition_data={}
for key,value in transition.items():
    for key1,value1 in value.items():
        transition_data[(key,key1)]=value1
        
        
emission={}
for i in corpus:
    for j in range(len(i)):
        if i[j][1] not in emission:
            emission[i[j][1]]={}
        if i[j][0] not in emission[i[j][1]]:
            emission[i[j][1]][i[j][0]]=0
        emission[i[j][1]][i[j][0]]+=1
        

for w1 in emission:
    tot = float(sum(emission[w1].values()))
    for w2 in emission[w1]:
        emission[w1][w2]=math.log2((emission[w1][w2]+0.001)/(0.001*len(word_dict) + tag_dict[w1]))

After that, we could start to process the POS Tagging algorithm using HMM. Given a sequence of words to be tagged, the task is to assign the most probable tag to the word. In other words, to every word w, assign the tag t that maximize the likelihood P(t/w). 

Since P(t/w) = P(w/t). P(t) / P(w), after ignoring P(w), we have to compute P(w/t) and P(t). 

As a result: 
P(w/t): is the emission probability of a given word for a given tag. This can be computed based on the fraction of given word for given tag to the total count of that tag, ie: P(w/t) = count(w, t) / count(t). 

P(t): is the probability of tag (also transition probability), and in a tagging task, we assume that a tag will depend only on the previous tag (Markov order 1 assumption). In other words, the probability of telling a tag being NN will depend only on the previous tag t(n-1). For example, if t(n-1) is a JJ, then t(n) is likely to be an NN since adjectives often precede a noun (blue coat, tall building etc.).

In [5]:
'''--------------------Viterbi Algorithm--------------------------------'''


test_data= brown.tagged_sents(tagset='universal')[-10:]
sentences =[]
for i in test_data:
    temp = [j[0] for j in i]
    sentences.append(temp)

average = 0    
seq = []
def find_max(i,dic):
    maxi = -999999
    
    tag=""
    for key,value in dic.items():
        if dic[key][i] > maxi:
            maxi = dic[key][i]
            tag = key
    return maxi,tag

for sent in range(len(sentences)):
    test = []
    dic={}  
    flag = 0
    for i in list(tag_dict.keys()):
        dic[i]=[]

    
    for i in sentences[sent]:
        if flag==0:
            for j in list(tag_dict.keys()):
                try:
                    prob = emission[j][i]+start[j]
                except Exception as e:
                    temp1 = math.log2(0.001/(0.001*len(word_dict) + tag_dict[j]))
                    prob = start[j] + temp1
                dic[j].append(prob)
        break
    
    max_prob, tag = find_max(0,dic)
    test.append(tag)
    
   
    
    
    flag = 0
    for i,w1 in enumerate(sentences[sent]):
        if i==0:
            continue
        for j in list(tag_dict.keys()):
            try:
                temp = transition[tag][j]
            except Exception as e:
                temp = math.log2((0.001/(0.001*len(word_dict) +float(sum(transition1[tag].values())))))
            try:
                prob_w2 = max_prob + emission[j][w1] + temp
            except Exception as e:
                temp1 = math.log2(0.001/(0.001*len(word_dict) + tag_dict[j]))
                prob_w2 = max_prob + temp1 + temp 
            dic[j].append(prob_w2)
            
        max_prob, tag = find_max(i,dic)
        test.append(tag)
    
    seq.append(test)
            
        
actual_tag=[]
for sent in test_data:
    temp=[]
    for word in sent:
        temp.append(word[1])
    actual_tag.append(temp)

Viterbi algorithm The steps are as follows: 
1. Given a sequence of words 

2. iterate through the sequence 

3. for each word (starting from first word in sequence) calculate the product of emission probabilities and transition probabilities for all possible tags. 

4. assign the tag which has maximum probability obtained in step 3 above. 

5. move to the next word in sequence to repeat steps 3 and 4 above.

In [6]:
'''-------------------------------CRF----------------------------'''

train_sents= corpus

def word2features(sent,i):
    word = sent[i][0]
    
    features ={
    'bias': 1.0,
    'word.lower()': word.lower(),
    'word[-3:]': word[-3:],
    'word[-2:]': word[-2:],
    'word.isupper()': word.isupper(),
    'word.istitle()': word.istitle(),
    'word.isdigit()': word.isdigit(),
    }
    if i > 0:
        word_prev = sent[i-1][0]
        features.update({
            '-1:word.lower()': word_prev.lower(),
            '-1:word.istitle()': word_prev.istitle(),
            '-1:word.isupper()': word_prev.isupper(),
        })
    else:
        features['start_sentence'] = True

    if i < len(sent)-1:
        word_after = sent[i+1][0]
        features.update({
            '+1:word.lower()': word_after.lower(),
            '+1:word.istitle()': word_after.istitle(),
            '+1:word.isupper()': word_after.isupper(),
        })
    else:
        features['end_sentence'] = True
                
    return features

def sent2features(sent):
    return [word2features(sent,i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for i,label in sent]


X_train=[sent2features(s) for s in train_sents]
y_train=[sent2labels(s) for s in train_sents]

X_test=[sent2features(s) for s in test_data]
y_test=[sent2labels(s) for s in test_data]


crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

y_pred = crf.predict(X_test)
labels=list(crf.classes_)

sorted_labels = sorted(
    labels, 
    key=lambda name: (name[1:], name[0])
)

The next step is to use the sklearn_crfsuite to fit the CRF model. The model is optimised by Gradient Descent using the LBGS method with L1 and L2 regularisation. We will set the CRF to generate all possible label transitions, even those that do not occur in the training data.

In [7]:

print('Number of test sentences used = 10')
print('----------------------Viterbi Results---------------------------')
print('Viterbi Accuracy Score :',metrics.flat_f1_score(actual_tag, seq,average='weighted', labels=labels))
print(metrics.flat_classification_report(
    actual_tag, seq, labels=sorted_labels, digits=3
))
print('------------------------CRF Results-----------------------------')
print('CRF Accuracy Score :',metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=labels))
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

Number of test sentences used = 10
----------------------Viterbi Results---------------------------


C:\Users\TK\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\TK\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Viterbi Accuracy Score : 0.9246991839814064


C:\Users\TK\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\TK\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


              precision    recall  f1-score   support

           .      0.943     1.000     0.971        33
           X      1.000     0.333     0.500         3
         ADJ      0.941     0.889     0.914        18
         ADP      0.962     0.926     0.943        27
         ADV      0.800     0.889     0.842         9
        VERB      0.941     0.914     0.928        35
         DET      1.000     1.000     1.000        33
        CONJ      0.636     1.000     0.778         7
        NOUN      0.978     0.863     0.917        51
        PRON      0.917     0.917     0.917        12
         PRT      0.733     1.000     0.846        11
         NUM      0.000     0.000     0.000         0

   micro avg      0.925     0.925     0.925       239
   macro avg      0.821     0.811     0.796       239
weighted avg      0.935     0.925     0.925       239

------------------------CRF Results-----------------------------
CRF Accuracy Score : 0.9570914166105637
              precision    r

Evaluating the HHM Model and CRF Model